In [ ]:
# Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas Series and 
# DataFrame objects, respectively. Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed 
# by more than one or two keys. While Pandas does provide Panel and Panel4D objects that natively handle three-dimensional and 
# four-dimensional data (see Aside: Panel Data), a far more common pattern in practice is to make use of hierarchical indexing 
# (also known as multi-indexing) to incorporate multiple index levels within a single index. In this way, higher-dimensional 
# data can be compactly represented within the familiar one-dimensional Series and two-dimensional DataFrame objects.

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# A Multiply Indexed Series

In [ ]:
# considering how we might represent two-dimensional data within a one-dimensional Series. 
# For concreteness, we will consider a series of data where each point has a character and numerical key.

In [ ]:
# the bad way:
# Suppose you would like to track data about states from two different years. Using the Pandas tools we've already covered, 
# you might be tempted to simply use Python tuples as keys:

In [4]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [5]:
# With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [ ]:
# But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy 
# (and potentially slow) munging to make it happen:

In [7]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

In [11]:
# this produces the right result but is not as clean and efficient for large datasets as the slicing syntax we use in pandas

In [ ]:
# The Better Way: Pandas MultiIndex

# Fortunately, Pandas provides a better way. Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas 
# MultiIndex type gives us the type of operations we wish to have. We can create a multi-index from the tuples as follows:

In [12]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [ ]:
# Notice that the MultiIndex contains multiple levels of indexing–in this case, the state names and the years, as well as 
# multiple labels for each data point which encode these levels.

In [14]:
# If we re-index our series with this MultiIndex, we see the hierarchical representation of the data:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
# Here the first two columns of the Series representation show the multiple index values, while the third column shows the data.
# Notice that some entries are missing in the first column: in this multi-index representation, any blank entry indicates the 
# same value as the line above it.

In [15]:
# Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:

pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [ ]:
# The result is a singly indexed array with just the keys we're interested in. This syntax is much more convenient 
# (and the operation is much more efficient!)

In [ ]:
# MultiIndex as extra dimension

In [17]:
# You might notice something else here: we could easily have stored the same data using a simple DataFrame with index and 
# column labels. In fact, Pandas is built with this equivalence in mind. The unstack() method will quickly convert a multiply 
# indexed Series into a conventionally indexed DataFrame:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [18]:
# Naturally, the stack() method provides the opposite operation:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
# Seeing this, you might wonder why would we would bother with hierarchical indexing at all. The reason is simple: 
# just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional Series, 
# we can also use it to represent data of three or more dimensions in a Series or DataFrame.

In [ ]:
# Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much 
# more flexibility in the types of data we can represent.

In [21]:
# Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18); 
# with a MultiIndex this is as easy as adding another column to the DataFrame:

pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [24]:
# In addition, all the ufuncs and other functionality discussed in Operating on Data in Pandas work with hierarchical indices 
# as well. Here we compute the fraction of people under 18 by year, given the above data:

f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [ ]:
# This allows us to easily and quickly manipulate and explore even high-dimensional data.

In [ ]:
# Methods of MultiIndex Creation

In [35]:
# The most straightforward way to construct a multiply indexed Series or DataFrame is 
# to simply pass a list of two or more index arrays to the constructor. For example:

df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.084870  0.711354
  2  0.128835  0.531355
b 1  0.900225  0.776093
  2  0.254442  0.802984

In [38]:
# Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a
# MultiIndex by default:

data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

In [ ]:
# Nevertheless, it is sometimes useful to explicitly create a MultiIndex; we'll see a couple of these methods here.

In [ ]:
# Explicit MultiIndex constructors

In [40]:
# For more flexibility in how the index is constructed, you can instead use the class method constructors available in the 
# pd.MultiIndex. For example, as we did before, you can construct the MultiIndex from a simple list of arrays giving the index
# values within each level:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 3]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 3)],
           )

In [41]:
# You can construct it from a list of tuples giving the multiple index values of each point:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [42]:
# You can even construct it from a Cartesian product of single indices:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [45]:
# Similarly, you can construct the MultiIndex directly using its internal encoding by passing levels 
# (a list of lists containing available index values for each level) and codes (a list of lists that reference these labels):
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [ ]:
# Any of these objects can be passed as the index argument when creating a Series or Dataframe, or be passed to the 
# reindex method of an existing Series or DataFrame.

In [46]:
# MultiIndex level names

# Sometimes it is convenient to name the levels of the MultiIndex. This can be accomplished by passing the names argument to 
# any of the above MultiIndex constructors, or by setting the names attribute of the index after the fact:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
# MultiIndex for columns

# In a DataFrame, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, 
# the columns can have multiple levels as well. Consider the following, which is a mock-up of some (somewhat realistic) medical
# data:

In [47]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      56.0  36.9  47.0  36.3  44.0  36.4
     2      33.0  38.5  34.0  36.7  33.0  37.1
2014 1      30.0  35.9  53.0  35.8  21.0  37.9
     2       7.0  37.1  63.0  37.5  44.0  36.0

In [ ]:
# Here we see where the multi-indexing for both rows and columns can come in very handy. This is fundamentally four-dimensional
# data, where the dimensions are the subject, the measurement type, the year, and the visit number. With this in place we can, 
# for example, index the top-level column by the person's name and get a full DataFrame containing just that person's 
# information:

In [48]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      47.0  36.3
     2      34.0  36.7
2014 1      53.0  35.8
     2      63.0  37.5

In [ ]:
# For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, 
# cities, etc.) use of hierarchical rows and columns can be extremely convenient!

In [ ]:
# Indexing and Slicing a MultiIndex

# Indexing and slicing on a MultiIndex is designed to be intuitive, and it helps if you think about the indices as added 
# dimensions. We'll first look at indexing multiply indexed Series, and then multiply-indexed DataFrames.

In [49]:
# Multiply indexed Series

# Consider the multiply indexed Series of state populations we saw earlier:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [50]:
# We can access single elements by indexing with multiple terms:

pop['California', 2000]

33871648

In [51]:
# The MultiIndex also supports partial indexing, or indexing just one of the levels in the index. 
# The result is another Series, with the lower-level indices maintained:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [52]:
# Partial slicing is available as well, as long as the MultiIndex is sorted (see discussion in Sorted and Unsorted Indices):
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [53]:
# With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [54]:
# Other types of indexing and selection (discussed in Data Indexing and Selection) work as well; for example, selection based 
# on Boolean masks:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [55]:
# Selection based on fancy indexing also works:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
# Multiply indexed DataFrames

In [56]:
# A multiply indexed DataFrame behaves in a similar manner. Consider our toy medical DataFrame from before:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      56.0  36.9  47.0  36.3  44.0  36.4
     2      33.0  38.5  34.0  36.7  33.0  37.1
2014 1      30.0  35.9  53.0  35.8  21.0  37.9
     2       7.0  37.1  63.0  37.5  44.0  36.0

In [57]:
# Remember that columns are primary in a DataFrame, and the syntax used for multiply indexed Series applies to the columns. 
# For example, we can recover Guido's heart rate data with a simple operation:
health_data['Guido', 'HR']

year  visit
2013  1        47.0
      2        34.0
2014  1        53.0
      2        63.0
Name: (Guido, HR), dtype: float64

In [58]:
# Also, as with the single-index case, we can use the loc, iloc indexers introduced in Data Indexing and Selection. 
# For example:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      56.0  36.9
     2      33.0  38.5

In [59]:
# These indexers provide an array-like view of the underlying two-dimensional data, but each individual 
# index in loc or iloc can be passed a tuple of multiple indices. For example:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        56.0
      2        33.0
2014  1        30.0
      2         7.0
Name: (Bob, HR), dtype: float64

In [60]:
# Working with slices within these index tuples is not especially convenient; trying to create a slice within a 
# tuple will lead to a syntax error:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-60-6633b479cf96>, line 3)

In [61]:
# You could get around this by building the desired slice explicitly using Python's built-in slice() function, 
# but a better way in this context is to use an IndexSlice object, which Pandas provides for precisely this situation. 
# For example:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,56.0,47.0,44.0
2014,1,30.0,53.0,21.0


In [ ]:
# Rearranging Multi-Indices

# One of the keys to working with multiply indexed data is knowing how to effectively transform the data. There are a number of
# operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations. 

In [62]:
# Sorted and unsorted indices
# Earlier, we briefly mentioned a caveat, but we should emphasize it more here. Many of the MultiIndex slicing operations will 
# fail if the index is not sorted. Let's take a look at this here.

# We'll start by creating some simple multiply indexed data where the indices are not lexographically sorted:

index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.859682
      2      0.859550
c     1      0.934903
      2      0.685093
b     1      0.201693
      2      0.363719
dtype: float64

In [63]:
# If we try to take a partial slice of this index, it will result in an error:

try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [64]:
# Although it is not entirely clear from the error message, this is the result of the MultiIndex not being sorted. For various
# reasons, partial slices and other similar operations require the levels in the MultiIndex to be in sorted 
# (i.e., lexographical) order. Pandas provides a number of convenience routines to perform this type of sorting; examples are 
# the sort_index() and sortlevel() methods of the DataFrame. We'll use the simplest, sort_index(), here:
data = data.sort_index()
data

char  int
a     1      0.859682
      2      0.859550
b     1      0.201693
      2      0.363719
c     1      0.934903
      2      0.685093
dtype: float64

In [65]:
# With the index sorted in this way, partial slicing will work as expected:
data['a':'b']

char  int
a     1      0.859682
      2      0.859550
b     1      0.201693
      2      0.363719
dtype: float64

In [66]:
# Stacking and unstacking indices
# As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional 
# representation, optionally specifying the level to use:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [69]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [70]:
# The opposite of unstack() is stack(), which here can be used to recover the original series:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [71]:
# Index setting and resetting
# Another way to rearrange hierarchical data is to turn the INDEX LABELS into COLUMNS; this can be accomplished with the 
# reset_index method. Calling this on the population dictionary will result in a DataFrame with a state and year column holding
# the information that was formerly in the index. For clarity, we can optionally specify the name of the data for the column 
# representation:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [75]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [76]:
# Often when working with data in the real world, the raw input data looks like this and it's useful to build a MultiIndex from
# the column values. This can be done with the set_index method of the DataFrame, which returns a multiply indexed DataFrame:

pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [80]:
d = pd.DataFrame({'a': [1,2,3], 'b': [4,5,6], 'c': [7,8,9]})
d

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9


In [84]:
d.set_index(['a', 'b'])

,,c
a,b,
1,4,7
2,5,8
3,6,9


In [86]:
d.set_index(['a', 'b']).reset_index()
# In practice, I find this type of reindexing to be one of the more useful patterns when encountering real-world datasets.

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9


In [91]:
# Data Aggregations on Multi-Indices
# We've previously seen that Pandas has built-in data aggregation methods, such as mean(), sum(), and max(). For hierarchically
#indexed data, these can be passed a level parameter that controls which subset of the data the aggregate is computed on.

#For example, let's return to our health data:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      56.0  36.9  47.0  36.3  44.0  36.4
     2      33.0  38.5  34.0  36.7  33.0  37.1
2014 1      30.0  35.9  53.0  35.8  21.0  37.9
     2       7.0  37.1  63.0  37.5  44.0  36.0

In [92]:
# Perhaps we'd like to average-out the measurements in the two visits each year. We can do this by naming the index level we'd like to explore, in this case the year:

data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     44.5  37.7  40.5  36.50  38.5  36.75
2014     18.5  36.5  58.0  36.65  32.5  36.95

In [93]:
# By further making use of the axis keyword, we can take the mean among levels on the columns as well:

data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,41.166667,36.983333
2014,36.333333,36.700000


In [ ]:
# Thus in two lines, we've been able to find the average heart rate and temperature measured among all subjects in all visits 
# each year. This syntax is actually a short cut to the GroupBy functionality, which we will discuss in Aggregation and 
# Grouping. While this is a toy example, many real-world datasets have similar hierarchical structure.